# Active Multi-Information Source Bayesian Quadrature

## Overview

This notebook exemplifies the setup in [Gessner, Mahsereci, and Gonzalez, 2019](https://arxiv.org/abs/1903.11331).
The goal is to find the integral of an expensive function that comes with cheaper approximations that are correlated with the function of interest.

For basics on Bayesian quadrature (BQ), refer to the [BQ introductory notebook](./Emukit-tutorial-Bayesian-quadrature-introduction.ipynb). 

In [1]:
import numpy as np
import GPy

import emukit.examples.active_multi_source_bayesian_quadrature as msbq

from emukit.core.loop.user_function import MultiSourceFunctionWrapper

### Define test functions

In [2]:
def f1(x):
    return msbq.example_functions.forrester_high(x)

def f2(x):
    return msbq.example_functions.forrester_low(x)

In [3]:
msfw = MultiSourceFunctionWrapper([f1, f2])

### Initial data

In [4]:
X_init = np.zeros(shape=(8, 2))
X_init[:3,0] = np.asarray([0.78, 0.55, 0.2])
X_init[3:,0] = np.asarray([0.98, 0.62, 0.59, 0.45, 0.35])
X_init[3:,1] = 1.   # levels
Y_init = np.zeros(shape=(8, 1)); Y_init[:3,:] = f1(X_init[:3,:1]); Y_init[3:, :] = f2(X_init[3:,:1])

N_init = Y_init.size

In [5]:
msfw.evaluate(X_init), Y_init

([UserFunctionResult(X: [0.78 0.  ], Y: [-5.72816011], extra_outputs: {}),
  UserFunctionResult(X: [0.55 0.  ], Y: [0.87119732], extra_outputs: {}),
  UserFunctionResult(X: [0.2 0. ], Y: [-0.63972711], extra_outputs: {}),
  UserFunctionResult(X: [0.98 1.  ], Y: [17.29398228], extra_outputs: {}),
  UserFunctionResult(X: [0.62 1.  ], Y: [5.7651177], extra_outputs: {}),
  UserFunctionResult(X: [0.59 1.  ], Y: [5.9729904], extra_outputs: {}),
  UserFunctionResult(X: [0.45 1.  ], Y: [4.74143518], extra_outputs: {}),
  UserFunctionResult(X: [0.35 1.  ], Y: [3.50099335], extra_outputs: {})],
 array([[-5.72816011],
        [ 0.87119732],
        [-0.63972711],
        [17.29398228],
        [ 5.7651177 ],
        [ 5.9729904 ],
        [ 4.74143518],
        [ 3.50099335]]))

### Define integral bounds
The kernel needs to know the integral bounds, because this is where the integral is computed.

In [6]:
integral_bounds = msbq.integral_bounds.IntegralBounds(np.zeros(shape=(1,1)), np.ones(shape=(1,1)), 1)

### Kernel

In [7]:
B = GPy.kern.Coregionalize(input_dim=1, output_dim=2, rank=1)
kern = msbq.kernels.IntegrableRBF(input_dim=1, integral_bounds=integral_bounds) 

multi_output_kern = msbq.kernels.IntegrableTensorProductKernel(kernels=[kern, B])

display(multi_output_kern)

integrable_product_kernel.,value,constraints,priors
rbf_with_integral.variance,1.0,+ve,
rbf_with_integral.lengthscale,1.0,+ve,
coregion.W,"(2, 1)",,
coregion.kappa,"(2,)",+ve,


In [8]:
m = msbq.multi_source_bq.MultiSourceBayesianQuadrature(X_init, Y_init, multi_output_kern)